In [7]:
#데이터 핸들링 라이브러리
import pandas as pd
import numpy as np
#데이터 시각화 라이브러리
import seaborn as sns
import matplotlib.pyplot as plt
import matplotlib as mpl
plt.rcParams.update({'font.family':'AppleGothic'})
mpl.rc('axes', unicode_minus=False)
%config InlineBackend.figure_format = 'retina'


import sys
sys.path.append('/Users/wooseongkyun/코드_아카이브/프로젝트2_머신러닝과EDA/binomial_classification/preprocessing_data.py')

In [8]:
#기후 데이터를 가져오기
weather_df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')
weather_df.drop(['강수확률(%)','발표시각','신뢰도'],axis=1,inplace=True)

#각 지역별 면적 데이터 가져오기
area_proportion_df=pd.read_csv('./시도별_지역면적.csv',encoding='euc-kr')
area_proportion_df=area_proportion_df.iloc[16:]
area_proportion_df['2020'] = area_proportion_df['2020'].astype(float)


area_proportion_df['2020.1']=area_proportion_df['2020.1'].astype(float) *1/100
area_proportion_df.drop('2020',axis=1,inplace=True)
area_proportion_df.set_index('남북한별 ',inplace=True)
#
#
#기후데이터에서 지역정보 확인하기
weather_df['지역'].value_counts()
#weathder_area=['서울.인천.경기','충청북도','충청남도','강원영서','강원영동','전라북도','전라남도','제주도','경상북도','경상남도']

#기후데이터 카테고리에 맞게 각 지역별 면적 비율 계산하기
seoul_incheon_gyeonggi= float( area_proportion_df.loc[['서울특별시','인천광역시','경기도']].sum().values)
chungbuk= area_proportion_df.loc['충청북도'].sum()
chungnam_daejeon_sejong= float(area_proportion_df.loc[['충청남도','대전광역시','세종특별자치시']].sum().values)
jeonbuk= area_proportion_df.loc['전라북도'].sum()
jeonnam_gwangju= float(area_proportion_df.loc[['전라남도','광주광역시']].sum().values)
gyeongbuk= area_proportion_df.loc['경상북도'].sum()
gyeongnam_busan_ulsan= float(area_proportion_df.loc[['경상남도','부산광역시','울산광역시']].sum().values)
jeju=  area_proportion_df.loc['제주특별자치도'].sum()


/var/folders/m9/0vsmjxhd7q30k5mzw0bk58hh0000gn/T/ipykernel_67650/2234861246.py:2: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  weather_df= pd.read_csv('./기상청_중기예보_최근5년데이터.csv',encoding='euc-kr')


In [9]:
weather_df

,지역,예보시각,예보
0,서울.인천.경기,2017-01-04 오전,맑음
1,서울.인천.경기,2017-01-04 오후,맑음
2,서울.인천.경기,2017-01-05 오전,구름조금
3,서울.인천.경기,2017-01-05 오후,구름조금
4,서울.인천.경기,2017-01-06 오전,구름많음
...,...,...,...
474755,경상남도,2022-01-07 오전,맑음
474756,경상남도,2022-01-07 오후,맑음
474757,경상남도,2022-01-08 오전,맑음
474758,경상남도,2022-01-09 오전,맑음


In [10]:
#강원도를 시도가 아닌 영동,영서지방으로 구분지었다. 영동,영서를 지리적 구분으로 정의한 데이터값을 불러오면
yeongdong_data= pd.read_csv('./영동지방.csv',encoding='euc-kr')
yeongdong_data=  yeongdong_data[['시군구(1)','2019']]
yeongdong_data.set_index('시군구(1)',inplace=True)
yeongdong_data.drop(['시군구(1)','시군구(1)'],axis=0,inplace=True)
yeongdong_data['2019'] = yeongdong_data['2019'].astype(float)
yeongdong_area = yeongdong_data.sum().values

yeongseo_data= pd.read_csv('./영서지방.csv',encoding='euc-kr')
yeongseo_data=  yeongseo_data[['시군구(1)','2019']]
yeongseo_data.set_index('시군구(1)',inplace=True)
yeongseo_data.drop(['시군구(1)','시군구(1)'],axis=0,inplace=True)
yeongseo_data['2019'] = yeongseo_data['2019'].astype(float)
yeongseo_area = yeongseo_data.sum().values

#위의 값을 이용하여 영동과 영서 면적 비율 계산하기
yeongdong= float((area_proportion_df.loc['강원도']*(yeongdong_area)/(yeongdong_area+yeongseo_area)).values)
yeongseo= float((area_proportion_df.loc['강원도']*(yeongseo_area)/(yeongdong_area+yeongseo_area)).values)

In [11]:
area_df= pd.DataFrame([seoul_incheon_gyeonggi,chungbuk,chungnam_daejeon_sejong,
                       yeongseo,yeongdong,jeonbuk,jeonnam_gwangju,
                       jeju,gyeongbuk,gyeongnam_busan_ulsan],
                      ['서울.인천.경기','충청북도','충청남도','강원영서','강원영동','전라북도',
                       '전라남도','제주도','경상북도','경상남도'])
area_df.reset_index(inplace=True)
area_df.rename({'index':'지역',0:'상대넓이'},axis=1,inplace=True)
area_df

,지역,상대넓이
0,서울.인천.경기,0.119000
1,충청북도,0.074000
2,충청남도,0.092000
3,강원영서,0.124035
4,강원영동,0.043965
5,전라북도,0.080000
6,전라남도,0.128000
7,제주도,0.018000
8,경상북도,0.190000
9,경상남도,0.124000


## 갑자기 든 생각인데 날씨에 대한 영향력을 고려한다면 지역 면적이 아니라 각 지역의 교통량이 가중치가 되어야 하는 것이 아닐까?
- 서울시는 매우 작은 영역을 차지하지만 거기에서 이동하는 인구는 몹시 큰것을 생각해봐도 그럿듯이
- 일단은 다음에..

In [12]:
area_proportion_df

,2020.1
남북한별,
서울특별시,0.006
부산광역시,0.008
대구광역시,0.009
인천광역시,0.011
광주광역시,0.005
대전광역시,0.005
울산광역시,0.011
세종특별자치시,0.005
경기도,0.102


In [13]:
#pd.merge를 통하여 결합시키고. '지역'컬럼 값을 삭제한다
df= pd.merge(weather_df,area_df,how='inner',on='지역')
df.drop('지역',axis=1,inplace=True)
df

,예보시각,예보,상대넓이
0,2017-01-04 오전,맑음,0.119
1,2017-01-04 오후,맑음,0.119
2,2017-01-05 오전,구름조금,0.119
3,2017-01-05 오후,구름조금,0.119
4,2017-01-06 오전,구름많음,0.119
...,...,...,...
474755,2022-01-07 오전,맑음,0.124
474756,2022-01-07 오후,맑음,0.124
474757,2022-01-08 오전,맑음,0.124
474758,2022-01-09 오전,맑음,0.124


In [14]:
AM_PM_lst=[]
day_lst=[]

date_lst=df['예보시각']
for date in date_lst:
    splited_date= date.split()
    day_lst.append(splited_date[0])
    AM_PM_lst.append(splited_date[-1])

df['예보일']=day_lst
df['오전_오후']=AM_PM_lst

#오전오후를 0과 1값으로 표현하기
is_PM =df['오전_오후']=='오후'
df['오전0_오후1']=is_PM
df['오전0_오후1']= df['오전0_오후1'].astype(int)

#중복된 데이터정보를 나타내는 열들 삭제하기
df.drop(['예보시각','오전_오후'],axis=1,inplace=True)
df.sort_values(by='예보일',ascending=True)

,예보,상대넓이,예보일,오전0_오후1
0,맑음,0.119,2017-01-04,0
94953,맑음,0.092,2017-01-04,1
237381,맑음,0.080,2017-01-04,1
237393,맑음,0.080,2017-01-04,0
237394,맑음,0.080,2017-01-04,1
...,...,...,...,...
332331,구름많음,0.128,2022-01-10,0
379794,구름많음,0.018,2022-01-10,0
379807,흐림,0.018,2022-01-10,0
427283,맑음,0.190,2022-01-10,0


In [34]:
#기상 관측 분류를 원-핫 인코딩하기
features=['예보']
encoded_df= pd.get_dummies(data=df,columns=features,prefix=features)
encoded_df


#각 지역별 관측된 기상현상값들을 동일한 가중치(1)로 두고 더하여 전국 날씨값을 계산하기
encoded_df2= encoded_df.copy()
encoded_df2['맑음']= encoded_df2['예보_맑음']+encoded_df2['예보_구름조금']+encoded_df2['예보_구름많음']
encoded_df2['비']=encoded_df2['예보_구름많고 비']+encoded_df2['예보_구름많고 소나기']+\
                  encoded_df2['예보_흐리고 비']+encoded_df2['예보_흐리고 소나기']
encoded_df2['흐림']=encoded_df2['예보_흐림']
encoded_df2['눈']=encoded_df2['예보_구름많고 눈']+encoded_df2['예보_구름많고 눈/비']+\
                  encoded_df2['예보_구름많고 비/눈']+encoded_df2['예보_흐리고 눈']+\
                  encoded_df2['예보_흐리고 눈/비']+encoded_df2['예보_흐리고 비/눈']

encoded_df2['비/눈']=encoded_df2['눈']+encoded_df2['비']
encoded_df2= encoded_df2[['예보일','맑음','흐림','비/눈','상대넓이']]
encoded_df2.groupby(by='상대넓이').value_counts()

상대넓이   예보일         맑음  흐림  비/눈
0.018  2021-05-09  1   0   0      26
       2021-04-14  1   0   0      26
       2021-04-15  1   0   0      26
       2021-04-16  1   0   0      26
       2021-04-18  1   0   0      26
                                  ..
0.190  2020-04-12  0   1   0       1
       2020-05-07  0   0   1       1
       2020-05-02  0   1   0       1
       2020-04-27  0   0   1       1
       2018-05-12  0   0   1       1
Length: 28253, dtype: int64

In [35]:
area_df

,지역,상대넓이
0,서울.인천.경기,0.119000
1,충청북도,0.074000
2,충청남도,0.092000
3,강원영서,0.124035
4,강원영동,0.043965
5,전라북도,0.080000
6,전라남도,0.128000
7,제주도,0.018000
8,경상북도,0.190000
9,경상남도,0.124000
